In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [76]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

 

def iou(y_true, y_pred):

    def f(y_true, y_pred):

        intersection = (y_true * y_pred).sum()

        union = y_true.sum() + y_pred.sum() - intersection

        x = (intersection + 1e-15) / (union + 1e-15)

        x = x.astype(np.float32)

        return x

    return tf.numpy_function(f, [y_true, y_pred], tf.float32)


smooth = 1e-15


def dice_coef(y_true, y_pred):

    y_true = tf.keras.layers.Flatten()(y_true)

    y_pred = tf.keras.layers.Flatten()(y_pred)

    intersection = tf.reduce_sum(y_true * y_pred)

    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

 
def dice_loss(y_true, y_pred):

    return 1.0 - dice_coef(y_true, y_pred)

In [77]:
#model

from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input, Dropout
from keras.layers import Input, Activation, Conv2D, MaxPooling2D, UpSampling2D, Dropout,BatchNormalization, AveragePooling2D
from tensorflow.keras.models import Model

 
def mnet(input_shape):

    # Input
    input1 = Input(input_shape)


    # Encoder
    conv1 = Conv2D(64, (1, 1), padding='same')(input1 )
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)
    conv1 = Dropout(0.2)(conv1)

    conv1 = Concatenate()([input1 , conv1])
    conv1 = Conv2D(64, (1, 1), padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)
    pool1 = MaxPool2D(pool_size=(2, 2))(conv1)




    #
    input2 = MaxPool2D(pool_size=(2, 2))(input1)
    conv21 = Concatenate()([input2 , pool1])
    conv2 = Conv2D(128, (1, 1), padding='same')(conv21)
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('relu')(conv2)
    conv2 = Dropout(0.2)(conv2)

    conv2 = Concatenate()([conv21, conv2])
    conv2 = Conv2D(128, (1, 1), padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('relu')(conv2)
    pool2 = MaxPool2D(pool_size=(2, 2))(conv2)


    #

    input3 = MaxPool2D(pool_size=(2, 2))(input2)


    conv31 = Concatenate()([input3 , pool2])
    conv3 = Conv2D(256, (1, 1), padding='same')(conv31 )
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)
    conv3 = Dropout(0.2)(conv3)

    conv3 = Concatenate()([conv31, conv3])
    conv3 = Conv2D(256, (1, 1), padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)
    pool3 = MaxPool2D(pool_size=(2, 2))(conv3)





    #
    input4 = MaxPool2D(pool_size=(2, 2))(input3)


    conv41 = Concatenate()([input4 , pool3])
    conv4 = Conv2D(512, (1, 1), padding='same')(conv41 )
    conv4 = BatchNormalization()(conv4)
    conv4 = Activation('relu')(conv4)
    conv4 = Dropout(0.2)(conv4)

    conv4 = Concatenate()([conv41, conv4])
    conv4 = Conv2D(512, (1, 1), padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    conv4 = Activation('relu')(conv4)
    pool4 = MaxPool2D(pool_size=(2, 2))(conv4)


    #
    input5 = MaxPool2D(pool_size=(2, 2))(input4)


    conv51 = Concatenate()([input5 , pool4])
    conv5 = Conv2D(1024, (1, 1), padding='same')(conv51)
    conv5 = BatchNormalization()(conv5)
    conv5 = Activation('relu')(conv5)
    conv5 = Dropout(0.2)(conv5)

    conv5 = Concatenate()([conv51, conv5])
    conv5 = Conv2D(1024, (1, 1), padding='same')(conv5)
    conv5 = BatchNormalization()(conv5)
    conv5 = Activation('relu')(conv5)

 
    # Decoder
    conv6 = UpSampling2D(size=(2, 2))(conv5)

    conv61 = Concatenate()([conv4, conv6])
    #conv61 = cbam_block(conv61)
    conv6 = Conv2D(512, (3, 3), padding='same')(conv61)
    conv6 = BatchNormalization()(conv6)
    conv6 = Activation('relu')(conv6)
    conv6 = Dropout(0.2)(conv6)

    conv6 = Concatenate()([conv61, conv6])
    conv6 = Conv2D(512, (3, 3), padding='same')(conv6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Activation('relu')(conv6)
    conv62 = Concatenate()([conv6, conv61])

    
    #
    conv7 = UpSampling2D(size=(2, 2))(conv6)
    conv71 = Concatenate()([conv3, conv7])
    #conv71 = cbam_block(conv71)
    conv7 = Conv2D(256, (3, 3), padding='same')(conv71)
    conv7 = BatchNormalization()(conv7)
    conv7 = Activation('relu')(conv7)
    conv7 = Dropout(0.2)(conv7)

    conv7 = Concatenate()([conv71, conv7])
    conv7 = Conv2D(256, (3, 3), padding='same')(conv7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Activation('relu')(conv7)
    conv72 = Concatenate()([conv71, conv7])

    #
    conv8 = UpSampling2D(size=(2, 2))(conv7)
    conv81 = Concatenate()([conv2, conv8])
    #conv81 = cbam_block(conv81)
    conv8 = Conv2D(128, (3, 3), padding='same')(conv81)
    conv8 = BatchNormalization()(conv8)
    conv8 = Activation('relu')(conv8)
    conv8 = Dropout(0.2)(conv8)

    conv8 = Concatenate()([conv81, conv8])
    conv8 = Conv2D(128, (3, 3), padding='same')(conv8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Activation('relu')(conv8)
    conv82 = Concatenate()([conv81, conv8])
    

    """
    # Final
    conv101 = UpSampling2D(size=(8, 8))(conv6)
    conv102 = UpSampling2D(size=(4, 4))(conv7)
    conv103 = UpSampling2D(size=(2, 2))(conv8)
    #outputs = Conv2D(1, 1, activation='sigmoid')(conv10)
    """

    #
    conv9 = UpSampling2D(size=(2, 2))(conv8)
    conv91 = Concatenate()([conv1, conv9])
    #conv91 = cbam_block(conv91)
    conv9 = Conv2D(64, (3, 3), padding='same')(conv91)
    conv9 = BatchNormalization()(conv9)
    conv9 = Activation('relu')(conv9)
    conv9 = Dropout(0.2)(conv9)
    
    conv9 = Concatenate()([conv91, conv9])
    conv9 = Conv2D(64, (3, 3), padding='same')(conv9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Activation('relu')(conv9)



    #첫번째 다리
    conv111 = Concatenate()([conv6, conv62])
    conv112 = Concatenate()([conv7, conv72])
    conv113 = Concatenate()([conv8, conv82])

    
    #두번째 다리 추가
    conv211 = Concatenate()([conv61, conv111])
    conv212 = Concatenate()([conv71, conv112])
    conv213 = Concatenate()([conv81, conv113])

    conv211 = UpSampling2D(size=(8, 8))(conv211)
    conv212 = UpSampling2D(size=(4, 4))(conv212)
    conv213 = UpSampling2D(size=(2, 2))(conv213)
    conv12 = Concatenate()([conv211, conv212, conv213, conv9])


    outputs = Conv2D(1, 1, activation='sigmoid')(conv12)


    model = Model(input1, outputs)
    return model

In [45]:
#channel_attention

from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, Reshape, Dense, multiply, Permute, Concatenate, Conv2D, Add, Activation, Lambda
from tensorflow.keras import backend as K
from tensorflow.keras.activations import sigmoid
import tensorflow as tf

def channel_att(input_feature, ratio=8):
	
	channel_axis = 1 if K.image_data_format() == "channels_first" else -1
	channel = input_feature.shape[channel_axis]

	shared_layer_one = Dense(channel//ratio,
							 activation='relu',
							 kernel_initializer='he_normal',
							 use_bias=True,
							 bias_initializer='zeros')
	shared_layer_two = Dense(channel,
							 kernel_initializer='he_normal',
							 use_bias=True,
							 bias_initializer='zeros')
	
	avg_pool = GlobalAveragePooling2D()(input_feature)    
	avg_pool = Reshape((1,1,channel))(avg_pool)
	assert avg_pool.shape[1:] == (1,1,channel)
	avg_pool = shared_layer_one(avg_pool)
	assert avg_pool.shape[1:] == (1,1,channel//ratio)
	avg_pool = shared_layer_two(avg_pool)
	assert avg_pool.shape[1:] == (1,1,channel)
	
	max_pool = GlobalMaxPooling2D()(input_feature)
	max_pool = Reshape((1,1,channel))(max_pool)
	assert max_pool.shape[1:] == (1,1,channel)
	max_pool = shared_layer_one(max_pool)
	assert max_pool.shape[1:] == (1,1,channel//ratio)
	max_pool = shared_layer_two(max_pool)
	assert max_pool.shape[1:] == (1,1,channel)
	
	channel_feature = Add()([avg_pool,max_pool])
	channel_feature = Activation('sigmoid')(channel_feature)
	
	if K.image_data_format() == "channels_first":
		channel_feature = Permute((3, 1, 2))(channel_feature)
  
	channel_feature = multiply([input_feature, channel_feature])
	return channel_feature

In [73]:
#attention

from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, Reshape, Dense, multiply, Permute, Concatenate, Conv2D, Add, Activation, Lambda
from tensorflow.keras import backend as K
from tensorflow.keras.activations import sigmoid
import tensorflow as tf

def attach_attention_module(net, attention_module):
  if attention_module == 'se_block': # SE_block
    net = se_block(net)
  elif attention_module == 'cbam_block': # CBAM_block
    net = cbam_block(net)
  else:
    raise Exception("'{}' is not supported attention module!".format(attention_module))

  return net

def se_block(input_feature, ratio=8):
	"""Contains the implementation of Squeeze-and-Excitation(SE) block.
	As described in https://arxiv.org/abs/1709.01507.
	"""
	
	channel_axis = 1 if K.image_data_format() == "channels_first" else -1
	channel = input_feature.shape[channel_axis]

	se_feature = GlobalAveragePooling2D()(input_feature)
	se_feature = Reshape((1, 1, channel))(se_feature)
	assert se_feature.shape[1:] == (1,1,channel)
	se_feature = Dense(channel // ratio,
					   activation='relu',
					   kernel_initializer='he_normal',
					   use_bias=True,
					   bias_initializer='zeros')(se_feature)
	assert se_feature.shape[1:] == (1,1,channel//ratio)
	se_feature = Dense(channel,
					   activation='sigmoid',
					   kernel_initializer='he_normal',
					   use_bias=True,
					   bias_initializer='zeros')(se_feature)
	assert se_feature.shape[1:] == (1,1,channel)
	if K.image_data_format() == 'channels_first':
		se_feature = Permute((3, 1, 2))(se_feature)

	se_feature = multiply([input_feature, se_feature])
	return se_feature

def cbam_block(cbam_feature, ratio=8):
	"""Contains the implementation of Convolutional Block Attention Module(CBAM) block.
	As described in https://arxiv.org/abs/1807.06521.
	"""
	
	cbam_feature = channel_attention(cbam_feature, ratio)
	cbam_feature = spatial_attention(cbam_feature)
	return cbam_feature

def channel_attention(input_feature, ratio=8):
	
	channel_axis = 1 if K.image_data_format() == "channels_first" else -1
	channel = input_feature.shape[channel_axis]
	
	shared_layer_one = Dense(channel//ratio,
							 activation='relu',
							 kernel_initializer='he_normal',
							 use_bias=True,
							 bias_initializer='zeros')
	shared_layer_two = Dense(channel,
							 kernel_initializer='he_normal',
							 use_bias=True,
							 bias_initializer='zeros')
	
	avg_pool = GlobalAveragePooling2D()(input_feature)    
	avg_pool = Reshape((1,1,channel))(avg_pool)
	assert avg_pool.shape[1:] == (1,1,channel)
	avg_pool = shared_layer_one(avg_pool)
	assert avg_pool.shape[1:] == (1,1,channel//ratio)
	avg_pool = shared_layer_two(avg_pool)
	assert avg_pool.shape[1:] == (1,1,channel)
	
	max_pool = GlobalMaxPooling2D()(input_feature)
	max_pool = Reshape((1,1,channel))(max_pool)
	assert max_pool.shape[1:] == (1,1,channel)
	max_pool = shared_layer_one(max_pool)
	assert max_pool.shape[1:] == (1,1,channel//ratio)
	max_pool = shared_layer_two(max_pool)
	assert max_pool.shape[1:] == (1,1,channel)
	
	cbam_feature = Add()([avg_pool,max_pool])
	cbam_feature = Activation('sigmoid')(cbam_feature)
	
	if K.image_data_format() == "channels_first":
		cbam_feature = Permute((3, 1, 2))(cbam_feature)
	
	return multiply([input_feature, cbam_feature])

def spatial_attention(input_feature):
	kernel_size = 7
	
	if K.image_data_format() == "channels_first":
		channel = input_feature.shape[1]
		cbam_feature = Permute((2,3,1))(input_feature)
	else:
		channel = input_feature.shape[-1]
		cbam_feature = input_feature
	
	avg_pool = Lambda(lambda x: K.mean(x, axis=3, keepdims=True))(cbam_feature)
	assert avg_pool.shape[-1] == 1
	max_pool = Lambda(lambda x: K.max(x, axis=3, keepdims=True))(cbam_feature)
	assert max_pool.shape[-1] == 1
	concat = Concatenate(axis=3)([avg_pool, max_pool])
	assert concat.shape[-1] == 2
	cbam_feature = Conv2D(filters = 1,
					kernel_size=kernel_size,
					strides=1,
					padding='same',
					activation='sigmoid',
					kernel_initializer='he_normal',
					use_bias=False)(concat)	
	assert cbam_feature.shape[-1] == 1
	
	if K.image_data_format() == "channels_first":
		cbam_feature = Permute((3, 1, 2))(cbam_feature)
		
	return multiply([input_feature, cbam_feature])

In [78]:
#train

import os
#os.environ["TF_CPP_MIN_LOG_LEVEL"] = "0"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
#from model import build_unet
#from metrics import dice_coef, iou

H = 256
W = 256

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def load_data(dataset_path, split=0.2):
    #dataset_path = os.getcwd()
    dataset_path=dataset_path+"/drive/MyDrive/Colab Notebooks/dataset"
    print(dataset_path)
    images = sorted(glob(os.path.join(dataset_path, "images", "*.jpg")))
    masks = sorted(glob(os.path.join(dataset_path, "tooth-semantic-masks", "*.png")))

    test_size = int(len(images) * split)

    train_x, valid_x = train_test_split(images, test_size=test_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=test_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=test_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=test_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)  ## (H, W, 3)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x                                ## (256, 256, 3)

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (H, W)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)                    ## (256, 256)
    x = np.expand_dims(x, axis=-1)              ## (256, 256, 1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Folder for saving data """
    create_dir("files1")

    """ Hyperparameters """
    batch_size = 1
    lr = 1e-4 ## (0.0001)
    num_epoch = 1
    model_path = "files1/model.h5"
    csv_path = "files1/data.csv"

    """ Dataset : 60/20/20 """
    dataset_path = os.getcwd()
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)

    print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Valid: {len(valid_x)} - {len(valid_y)}")
    print(f"Test: {len(test_x)} - {len(test_y)}")

    train_dataset = tf_dataset(train_x, train_y, batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y, batch_size)

    train_steps = len(train_x)//batch_size
    valid_steps = len(valid_x)//batch_size

    if len(train_x) % batch_size != 0:
        train_steps += 1

    if len(valid_x) % batch_size != 0:
        valid_steps += 1

    """ Model """
    model = mnet((H, W, 3))
    metrics = [dice_coef, iou, Recall(), Precision()]
    model.compile(loss="binary_crossentropy", optimizer=Adam(lr), metrics=metrics)
    model.summary()

    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path),
        TensorBoard(),
        EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
    ]

    model.fit(
        train_dataset,
        epochs=num_epoch,
        validation_data=valid_dataset,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        callbacks=callbacks
    )

/content/drive/MyDrive/Colab Notebooks/dataset
Train: 900 - 900
Valid: 300 - 300
Test: 300 - 300
Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_17 (InputLayer)          [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_238 (Conv2D)            (None, 256, 256, 64  256         ['input_17[0][0]']               
                                )                                                                 
                                                                                                  
 batch_normalization_212 (Batch  (None, 256, 256, 64  256        ['conv2d_238[0][0]']         

In [79]:
#test

import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "0"

import numpy as np

import cv2

import pandas as pd

from glob import glob

from tqdm import tqdm

import tensorflow as tf

from tensorflow.keras.utils import CustomObjectScope

from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score

#from metrics import dice_loss, dice_coef, iou

#from train import load_data, create_dir

 

H = 256

W = 256

 

def read_image(path):

    x = cv2.imread(path, cv2.IMREAD_COLOR)  ## (H, W, 3)

    x = cv2.resize(x, (W, H))

    ori_x = x

    x = x/255.0

    x = x.astype(np.float32)

    x = np.expand_dims(x, axis=0)

    return ori_x, x                                ## (1, 256, 256, 3)

 

 

def read_mask(path):

    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (H, W)

    x = cv2.resize(x, (W, H))

    ori_x = x

    x = x/255.0

    x = x.astype(np.int32)                    ## (256, 256)

    return ori_x, x

 

def save_results(ori_x, ori_y, y_pred, save_image_path):

    line = np.ones((H, 10, 3)) * 255

 

    ori_y = np.expand_dims(ori_y, axis=-1)  ## (256, 256, 1)

    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1) ## (256, 256, 3)

 

    y_pred = np.expand_dims(y_pred, axis=-1)  ## (256, 256, 1)

    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) ## (256, 256, 3)

 

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred*255], axis=1)

    cv2.imwrite(save_image_path, cat_images)

 

def load_data(dataset_path, split=0.2):

    dataset_path=dataset_path+"/drive/MyDrive/Colab Notebooks/dataset"

    images = sorted(glob(os.path.join(dataset_path, "images", "*.jpg")))

    masks = sorted(glob(os.path.join(dataset_path, "tooth-semantic-masks", "*.png")))

 

    test_size = int(len(images) * split)

 

    train_x, valid_x = train_test_split(images, test_size=test_size, random_state=42)

    train_y, valid_y = train_test_split(masks, test_size=test_size, random_state=42)

 

    train_x, test_x = train_test_split(train_x, test_size=test_size, random_state=42)

    train_y, test_y = train_test_split(train_y, test_size=test_size, random_state=42)

 

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

 

 

if __name__ == "__main__":

    """ Seeding """

    np.random.seed(42)

    tf.random.set_seed(42)

 

    """ Folder for saving results """

    create_dir("results1")

 

    """ Load the model """

    with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef}):

        model = tf.keras.models.load_model("files1/model.h5")

 

    """ Load the test data """

    dataset_path = os.getcwd()

    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)

 

    SCORE = []

    for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):

        """ Exctracting the image name """

        name = x.split("/")[-1]

 

        """ Read the image and mask """

        ori_x, x = read_image(x)

        ori_y, y = read_mask(y)

 

        """ Predicting the mask """

        y_pred = model.predict(x)[0] > 0.5

        y_pred = np.squeeze(y_pred, axis=-1)

        y_pred = y_pred.astype(np.int32)

 

        """ Saving the predicted mask """

        save_image_path = f"results1/{name}"

        save_results(ori_x, ori_y, y_pred, save_image_path)

        print(save_image_path)

 

        """ Flatten the array """

        y = y.flatten()

        y_pred = y_pred.flatten()

 

        """ Calculating metrics values """

        acc_value = accuracy_score(y, y_pred)

        f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")

        jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")

        recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")

        precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary")

        SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value])

 

    """ mean metrics values """

    score = [s[1:] for s in SCORE]

    score = np.mean(score, axis=0)

    print(f"Accuracy: {score[0]:0.5f}")

    print(f"F1: {score[1]:0.5f}")

    print(f"Jaccard: {score[2]:0.5f}")

    print(f"Recall: {score[3]:0.5f}")

    print(f"Precision: {score[4]:0.5f}")

 

    df = pd.DataFrame(SCORE, columns = ["Image Name", "Acc", "F1", "Jaccard", "Recall", "Precision"])

    df.to_csv("files1/score.csv")

  0%|          | 1/300 [00:00<02:44,  1.82it/s]

results1/cate6-00093.jpg
results1/cate2-00058.jpg


  1%|          | 3/300 [00:00<01:17,  3.85it/s]

results1/cate3-00004.jpg
results1/cate2-00147.jpg


  2%|▏         | 5/300 [00:01<01:01,  4.81it/s]

...

results1/cate6-00056.jpg
results1/cate2-00217.jpg


 99%|█████████▉| 298/300 [00:53<00:00,  5.60it/s]

results1/cate7-00090.jpg
results1/cate9-00017.jpg


100%|██████████| 300/300 [00:53<00:00,  5.59it/s]

results1/cate4-00112.jpg
Accuracy: 0.90548
F1: 0.68880
Jaccard: 0.53422
Recall: 0.57611
Precision: 0.89486


In [ ]:
dataset_path=os.getcwd()
dataset_path=dataset_path+ "/content/drive/MyDrive/Colab Notebooks/dataset"

In [ ]:
import os
print(os.getcwd())

/content
